In [27]:
# Importing pre-requisite libraries for the project

import requests
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
from itertools import cycle
import traceback
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os
from dotenv import load_dotenv


# Setting up environment variables

load_dotenv()

True

In [28]:
# Initiate Selenium Framework (Chromium Headless Browser)

def initialize_chromium():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-infobars")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-extensions")
    options.add_argument('--window-size=1920,1080')
    # options.add_argument("--headless")
    global driver; driver = webdriver.Chrome(options = options)


# Release Selenium Framework (Chromium Headless Browser)

def release_chromium():
    driver.close()

In [29]:
# Extracting the Regular Meetings data to a DataFrame

def meetings_extraction():
  # url = 'https://ottawapoliceboard.ca/opsb-cspo/meetings.html'
  # url = 'https://pub-ottawa.escribemeetings.com/?Expanded=Ottawa%20Police%20Services%20Board&Year=2023'
  # url = 'https://pub-ottawa.escribemeetings.com/Meeting.aspx?Id=3760766e-baa1-4574-a3c3-da5e7879c8c6&Agenda=Agenda&lang=English'

  global current_year; current_year = 2022
  reg_meets_all = []
  reg_meets_cancelled = []
  reg_meets_without_verbals = []
  meeting_no = 1

  initialize_chromium()

  while current_year > 2010:
    url = f'https://ottawapoliceboard.ca/opsb-cspo/{current_year}-meetings.html'

    driver.get(url)
    page_src = driver.page_source
    soup = BeautifulSoup(page_src, 'html.parser')

    meetings_list = soup.find('table').find('tbody').find_all('tr')

    for index, tr in enumerate(reversed(meetings_list)):
        td = tr.find_all('td')
        if td[3].get_text().strip() == 'Regular Meeting':
          if current_year < 2013: date = f'{td[0].get_text().strip()}, {current_year}'
          else: date = td[0].get_text().strip()

          # driver.get(td[0].find('a').get('href'))
          # page_src = driver.page_source

          reg_meets_all.append({
              'Meeting #': meeting_no,
              'Date': date,
              'Location': td[1].get_text().strip(),
              'Time': td[2].get_text().strip(),
              'Meeting Type': td[3].get_text().strip(),
              'Meeting Page': td[0].find('a').get('href')
            })
          meeting_no = meeting_no + 1
    print(f'Regular Meetings of Year {current_year} processed.')
    current_year = current_year - 1
  export_to_excel(reg_meets_all)


# Remove Cancelled Regular Meetings


# Check for Police Chief Verbals from the meetings list

def check_chief_verbals():
  print()


# Export the DataFrame to an Excel Sheet

def export_to_excel(df1, df2, df3):
  df1 = pd.DataFrame(df1)
  df1.to_excel('regular_meetings_v2.0.xlsx', sheet_name = 'All Regular Meetings')

In [30]:
def send_mail():
    recipient_mail = input("Enter an E-mail address to receive missing chief's verbal list")
    to_mail = recipient_mail
    gmail_usr = os.getenv('GMAIL_USR')
    gmail_pwd = os.getenv('GMAIL_PWD')

    msg = MIMEMultipart()
    msg['Subject'] = "Missing Chief's Verbals List"
    msg['From'] = gmail_usr
    msg['To'] = to_mail

    msgText = MIMEText("\n This mail consists of regular meetings list where Chief's Verbal document is missing \n p.f.a. \n", 'html')
    msg.attach(msgText)

    filename = "regular_meetings_v2.0.xlsx"
    xlsx = MIMEApplication(open(filename, 'rb').read())
    xlsx.add_header('Content-Disposition', 'attachment', filename = 'missing_chief_verbals.xlsx')
    msg.attach(xlsx)

    # with open(filename, 'rb') as f:
    #     file_data = f.read()
    # smtpserver.add_attachment(file_data, maintype="application", subtype="xlsx", filename='regular_meetings.xlsx')

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as smtpserver:
            smtpserver.ehlo()
            smtpserver.starttls()
            smtpserver.ehlo()
            smtpserver.login(gmail_usr, gmail_pwd)
            smtpserver.sendmail(f'{gmail_usr}<Team-8 SMTP Mail Client>', to_mail, msg.as_string())
            smtpserver.quit()
    except Exception as e:
        print(e)

    print ('email sent!')

In [31]:
meetings_extraction()
release_chromium()
# send_mail()

Regular Meetings of Year 2022 processed.
Regular Meetings of Year 2021 processed.
Regular Meetings of Year 2020 processed.
Regular Meetings of Year 2019 processed.
Regular Meetings of Year 2018 processed.
Regular Meetings of Year 2017 processed.
Regular Meetings of Year 2016 processed.
Regular Meetings of Year 2015 processed.
Regular Meetings of Year 2014 processed.
Regular Meetings of Year 2013 processed.
Regular Meetings of Year 2012 processed.
Regular Meetings of Year 2011 processed.


SMTPServerDisconnected: please run connect() first